# BCC Country preprocessing

In this notebook, we will add GDAM geometries to the Blue Carbon Cost Tool countries.

## Setup

### Library import

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon

import sys

sys.path.append("../src")

from gadm.preprocess_gadm import download_gadm_data, preprocess_gadm_data, write_sql_file

### Pipe

In [39]:
# Variables
folder_path = "./../raw_data"
import_xls_path = "./../excel/Carbon-Cost Data Upload.xlsm"
output_sql_path = "./../excel/insert_country_data.sql"

In [4]:
gpkg_path = download_gadm_data(folder_path)


Data unzipped in the folder: /home/alicitita/Projects/tnc-blue-carbon-cost-tool/data/raw_data/gadm


In [33]:
prepared_data = preprocess_gadm_data(gpkg_path, import_xls_path)

Saving the cleaned data to a csv in:  /home/alicitita/Projects/tnc-blue-carbon-cost-tool/data/raw_data/gadm/countries.csv


In [40]:
write_sql_file(prepared_data, output_sql_path)
